# Let's Fetch **The Price of Mobile Phones in India Today**

In [1]:
import pandas as pd
import json, time, random
from datetime import datetime
from pathlib import Path
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

In [2]:
options = Options()
options.add_argument("--blink-settings=imagesEnabled=false")
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.77 Safari/537.37")

current_date = datetime.now().strftime("%Y-%m-%d")
source_url = "https://www.sangeethamobiles.com/product-list/category-smartphones-308?sort_by=popularity"

driver = webdriver.Chrome(options=options)
driver.get(source_url)

In [3]:
for _ in range(2):
    load_more_button = driver.find_element(By.CSS_SELECTOR, "button.btn.btn-dark-custom-outline.btn__spacer-60")
    driver.execute_script("arguments[0].click();", load_more_button)
    time.sleep(3)

In [4]:
def parse_mobile_html(html_content):
    """Parse mobile phone HTML block into JSON using BeautifulSoup"""
    soup = BeautifulSoup(html_content, 'lxml')
    
    mobile_data = {}
    
    # Product name
    name_elem = soup.find('h2')
    mobile_data['name'] = name_elem.get_text(strip=True) if name_elem else ""
    
    # Product variant
    variant_elem = soup.find('p', class_='product-muted')
    mobile_data['variant'] = variant_elem.get_text(strip=True) if variant_elem else ""
    
    # Pricing
    new_price_elem = soup.find('div', class_='new-price-1')
    mobile_data['current_price'] = new_price_elem.get_text(strip=True) if new_price_elem else ""
    
    old_price_elem = soup.find('div', class_='old-price')
    mobile_data['original_price'] = old_price_elem.get_text(strip=True) if old_price_elem else ""
    
    discount_elem = soup.find('div', class_='price-discount')
    mobile_data['discount'] = discount_elem.get_text(strip=True) if discount_elem else ""
    
    # Delivery
    delivery_elem = soup.find('div', class_='offer-label')
    mobile_data['delivery_by'] = delivery_elem.get_text(strip=True) if delivery_elem else ""
    
    # URL
    link_elem = soup.find('a')
    mobile_data['url'] = link_elem.get('href', '') if link_elem else ""
    
    return mobile_data

In [7]:
mobiles = []
jace = driver.find_elements(By.CLASS_NAME, "product-details")

for n in range(len(jace)):
    mobile_json = parse_mobile_html(jace[n].get_attribute("innerHTML"))
#   mobile_json.pop("delivery_by", None)
    mobile_json.pop("variant", None)
    mobile_json.pop("url", None)
    mobile_json.pop("discount", None)
    mobile_json.pop("original_price", None)
    mobiles.append(mobile_json)
    mobile_json['isAvailable'] = False if mobile_json['delivery_by'] == '' else True
    mobile_json.pop("delivery_by", None)

df_mobiles = pd.DataFrame(mobiles)
df_mobiles.rename(columns={"name": "Phone", "current_price": "Price"}, inplace=True)
df_mobiles = df_mobiles[df_mobiles['isAvailable'] == True]
df_mobiles['Price'] = df_mobiles['Price'].str.replace('₹', '')
df_mobiles['Price'] = df_mobiles['Price'].astype(float)
df_mobiles['Price'] = df_mobiles['Price'].astype(int)
df_mobiles = df_mobiles.sort_values(by='Price', ascending=False)
df_mobiles = df_mobiles.reset_index(drop=True)

df_mobiles

,Phone,Price,isAvailable
0,Apple iPhone 16 Pro 1TB Natural Titanium,157900,True
1,Apple iPhone 16 Pro Max 256GB Black Titanium,132900,True
2,Apple iPhone 16 Pro 256GB White Titanium,117900,True
3,Apple iPhone 16 Plus 128GB Teal,81399,True
4,Samsung Galaxy S22 Phantom Black 8GB 256GB,63152,True
5,Vivo V40 Pro 12GB 512GB Ganges Blue,52677,True
6,Samsung Galaxy S24 FE 5G 8GB 256GB Graphite,48399,True
7,OnePlus 11 8GB 128GB Eternal Green,35999,True
8,Samsung Galaxy A54 5G Awesome Violet 8GB 128GB,33393,True
9,OnePlus Nord 2T 5G jade Fog 8GB 128GB,28999,True


In [9]:
stats = df_mobiles['Price'].describe().T

print("PRICES OF IN-STOCK MOBILE PHONES AS ON ", end='')
print(datetime.strptime(current_date, '%Y-%m-%d').strftime('%d %B, %Y').upper())
print(f"Source: {source_url}")
print("")
print(f"75% Percentile: ₹{int(stats['75%']):,}")
print(f"50% Percentile: ₹{int(stats['50%']):,}")
print(f"25% Percentile: ₹{int(stats['25%']):,}")

PRICES OF IN-STOCK MOBILE PHONES AS ON 27 JULY, 2025
Source: https://www.sangeethamobiles.com/product-list/category-smartphones-308?sort_by=popularity

75% Percentile: ₹49,468
50% Percentile: ₹20,134
25% Percentile: ₹14,776
